In [2]:
import pandas as pd
import requests
import json

## API AlphaVantage - Intradiario

In [1]:
with open("C:/Users/hvzam/pwd_coder.txt", "r") as f:
    password = f.read()
    
with open("C:/Users/hvzam/token_alphavantage_coder.txt", "r") as f:
    token = f.read()

In [3]:
def getIntra(symbol, interval, size):
    function = 'TIME_SERIES_INTRADAY'
    url = 'https://www.alphavantage.co/query'
    parametros = {'function': function, 'symbol': symbol, 'interval': interval, 'outputsize': size, 'datatype': json, 'apikey': token}
    
    r = requests.get(url, params=parametros)
    data = r.json()['Time Series ('+interval+')']
    dataDF = pd.DataFrame.from_dict(data, orient='index') # Convertir a DataFrame
    dataDF = dataDF.astype('float')
    dataDF.index.name = 'Fecha'
    dataDF.columns = ['Apertura', 'Maximo', 'Minimo', 'Cierre', 'Volumen']
    dataDF = dataDF.sort_values('Fecha', ascending=True)
    dataDF.index = pd.to_datetime(dataDF.index)
    return dataDF

acciones = ['AMZN', 'MELI', 'AAPL', 'GOOGL', 'PINS'] # Lista de símbolos de acciones que voy a consultar

combined_data = pd.DataFrame() # Inicializar un DataFrame vacío para combinar los datos

# Obtener y combinar los datos de las acciones en el DataFrame
for accion in acciones:
    data = getIntra(symbol=accion, interval='5min', size='compact')
    data['Simbolo'] = accion # Agregar una columna para el símbolo de la acción
    combined_data = pd.concat([combined_data, data])

combined_data = combined_data.round(3) # Redondear los valores a tres decimales

print(combined_data) # Imprimir el DataFrame combinado


                     Apertura   Maximo   Minimo   Cierre   Volumen Simbolo
Fecha                                                                     
2023-10-13 11:40:00   130.580  130.645  130.330  130.370  357837.0    AMZN
2023-10-13 11:45:00   130.370  130.400  130.200  130.275  293090.0    AMZN
2023-10-13 11:50:00   130.305  130.306  129.931  129.965  471138.0    AMZN
2023-10-13 11:55:00   129.955  129.980  129.810  129.880  821916.0    AMZN
2023-10-13 12:00:00   129.880  130.045  129.820  129.920  363596.0    AMZN
...                       ...      ...      ...      ...       ...     ...
2023-10-13 19:25:00    26.620   26.620   26.620   26.620     231.0    PINS
2023-10-13 19:30:00    26.610   26.620   26.610   26.620       3.0    PINS
2023-10-13 19:35:00    26.610   26.610   26.520   26.520     499.0    PINS
2023-10-13 19:45:00    26.520   26.520   26.520   26.520      50.0    PINS
2023-10-13 19:55:00    26.620   26.620   26.620   26.620      22.0    PINS

[500 rows x 6 columns]


In [14]:
import psycopg2

try:
    conn = psycopg2.connect(host="data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com", port = 5439, database="data-engineer-database", user="hvzambrana_coderhouse", password=password)
    print("Conexión a la base de datos exitosa")
except Exception as e:
    print("Falló al crear la conexión")
    print(e)

Conexión a la base de datos exitosa


In [15]:
try:
    cur = conn.cursor()
    cur.execute("""
    CREATE TABLE IF NOT EXISTS hvzambrana_coderhouse.alphavantage2 (
        simbolo VARCHAR(10) NULL,
        fecha TIMESTAMP NULL,
        apertura FLOAT(15) NULL,
        maximo FLOAT(15) NULL,
        minimo FLOAT(15) NULL,
        cierre FLOAT(15) NULL,
        volumen FLOAT(15) NULL
    );""")

except Exception as e:
    print("Falló al crear la tabla")
    print(e)

In [ ]:
for row in rows:
    conn.execute("""
    INSERT INTO hvzambrana_coderhouse.alphavantage(simbolo...) values(row["simbolo"]...)
                """)